This script was seeded by a block of code in Baffle_phase_space.ipynb .

Near field sources with names of the form NFS_3.* are constructed in poopy by propagating a gaussing beam at the lens output to the input of the tertiary. They are then imported into CST where they used to calculate primary illuminations among other things. 

In [1]:
# Uses nominal baffle design to generate NFS_3.1.3 (50GHz), NFS_3.2 (100GHz), NFS_3.3 (200GHz), NFS_3.4 (420GHz), , NFS_3.5 (300GHz)

import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy import constants as const
import poppy
from poppy.wfe import WavefrontError
from astropy.units import cds

fwhm_ = np.sqrt(np.log(2)/297.4178) * u.m 
lenslet_aperture = poppy.GaussianAperture(fwhm=fwhm_)
ff = 300*u.GHz # EXCLAIM Band = 420-540GHz
ll = const.c/ff

widths=[0.041*u.m] #u.Quantity(np.arange(0.03,0.05,0.001),u.m) #* u.m

#Radii
coldstop = poppy.CircularAperture(radius=3.81*u.cm)
baffle = poppy.CircularAperture(radius=3.874*u.cm) 


# separations 
d_lens_3 = 33.06 * u.cm

# position of baffles above lens in inches
baffle_z_array = [0.350,
        0.700,
        1.050,
        2.150,
        2.500,
        2.850,
        3.950,
        4.300,
        4.650]

for i,baffle_z in enumerate(baffle_z_array):
    baffle_z_array[i] = (baffle_z*u.cds.inch).to(u.cm)


#Trevor uses 8.13
wf = poppy.FresnelWavefront(beam_radius=10*u.cm, wavelength=ll) #npix=2048 etc 
    
#wf *= phase_gradient(4.5*u.mm,d_lenslet_lens,ff)

wf *= lenslet_aperture
    
wf *= coldstop

wf.propagate_fresnel(0.35*u.cm)
        
for i,baffle_z in enumerate(baffle_z_array):
    if i==0:
        wf.propagate_fresnel(baffle_z)
    else:
        wf.propagate_fresnel(baffle_z-baffle_z_array[i-1])
    wf*=baffle

wf.propagate_fresnel(25 * u.cm - baffle_z_array[-1]) #propagate to NFS

y, x = wf.coordinates()
E=np.sqrt(wf.intensity)
phase=wf.phase
ERe=np.multiply(E,np.cos(phase))
EIm=np.multiply(E,np.sin(phase))

xm=np.abs(x)<0.05
ym=np.abs(y)<0.05
mask=np.logical_and(xm,ym)

xfile = open("NFS_3.5_x.txt","w")
np.savetxt(xfile,x[mask])
xfile.close()

yfile = open("NFS_3.5_y.txt","w")
np.savetxt(yfile,y[mask])
yfile.close()

ERefile = open("NFS_3.5_ERe.txt","w")
np.savetxt(ERefile,ERe[mask])
ERefile.close()

EImfile = open("NFS_3.5_EIm.txt","w")
np.savetxt(EImfile,EIm[mask])
EImfile.close()